<a href="https://colab.research.google.com/github/Levelleor/googlecollab/blob/main/ComfyUIonColab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#`mountUmount(`<font size="3px" color="#01c968">`Gdrive`</font>`)`



In [1]:
#@markdown <br><center><img src='https://upload.wikimedia.org/wikipedia/commons/thumb/d/da/Google_Drive_logo.png/600px-Google_Drive_logo.png' height="50" alt="Gdrive-logo"/></center>
#@markdown <center><h3>Mount Gdrive to /content/drive</h3></center><br>
MODE = "MOUNT" #@param ["MOUNT", "UNMOUNT"]
#Mount your Gdrive!
from google.colab import drive
drive.mount._DEBUG = False
if MODE == "MOUNT":
  drive.mount('/content/drive', force_remount=True)
elif MODE == "UNMOUNT":
  try:
    drive.flush_and_unmount()
  except ValueError:
    pass
  get_ipython().system_raw("rm -rf /root/.config/Google/DriveFS")

Mounted at /content/drive


#`Setup And Update ComfyUI`



In [1]:
from pathlib import Path

OPTIONS = {}

DRIVE_PATH = ""  # @param {type:"string"}
UPDATE_COMFY_UI = True  #@param {type:"boolean"}
WORKSPACE = '/content/ComfyUI'
OPTIONS['UPDATE_COMFY_UI'] = UPDATE_COMFY_UI

if DRIVE_PATH:

    WORKSPACE = DRIVE_PATH+"/ComfyUI"
    %cd {DRIVE_PATH}

![ ! -d WORKSPACE ] && echo -= Initial setup ComfyUI =- && git clone https://github.com/comfyanonymous/ComfyUI
%cd $WORKSPACE

if OPTIONS['UPDATE_COMFY_UI']:
  !echo -= Updating ComfyUI =-
  !git pull

!echo -= Install dependencies =-
!pip install xformers!=0.0.18 -r requirements.txt --extra-index-url https://download.pytorch.org/whl/cu121 --extra-index-url https://download.pytorch.org/whl/cu118 --extra-index-url https://download.pytorch.org/whl/cu117

%cd {WORKSPACE}

-= Initial setup ComfyUI =-
Cloning into 'ComfyUI'...
remote: Enumerating objects: 29184, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 29184 (delta 17), reused 8 (delta 8), pack-reused 29154 (from 2)
Receiving objects: 100% (29184/29184), 75.64 MiB | 15.36 MiB/s, done.
Resolving deltas: 100% (19783/19783), done.
/content/ComfyUI
-= Updating ComfyUI =-
Already up to date.
-= Install dependencies =-
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu121, https://download.pytorch.org/whl/cu118, https://download.pytorch.org/whl/cu117
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of torchaudio to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 72.7 MB/s eta

# `Models Download`

## System Setup, API & Helper Functions

In [2]:
# --- 1. System Setup & Definitions ---
import os
import sys
from google.colab import userdata

# 1.1 Install System Dependencies
print("Installing System Dependencies...")
!apt-get -y install -qq aria2
!rm -rf /content/sample_data

# 1.2 Detect Environment
if os.path.exists('/content/ComfyUI/venv'):
    pip_cmd = '/content/ComfyUI/venv/bin/pip'
    py_cmd = '/content/ComfyUI/venv/bin/python'
elif os.path.exists('/content/venv'):
    pip_cmd = '/content/venv/bin/pip'
    py_cmd = '/content/venv/bin/python'
else:
    pip_cmd = 'pip'
    py_cmd = 'python'

# 1.3 Install Critical Python Libs (Required by Impact/SegmentAnything/etc)
print(f"Installing Python Dependencies using {pip_cmd}...")
!{pip_cmd} install -q segment-anything ultralytics scikit-image piexif timm transformers GitPython opencv-python-headless addict yapf onnxruntime rembg

# 1.4 Load Secrets
try:
  CIVITAI_API_TOKEN = userdata.get('CIVITAI_API_TOKEN')
except:
  CIVITAI_API_TOKEN = None
print("Civitai API Token:", "✅ Loaded" if CIVITAI_API_TOKEN else "❌ Not found")

# 1.5 Define The "Smart" Downloader
def smart_download(url, destination, filename=None, force=False):
    """
    Downloads files using aria2.
    - Handles Civitai Auth automatically.
    - Skips if file exists (unless force=True).
    - Handles custom filenames.
    """
    os.makedirs(destination, exist_ok=True)

    # Handle Filename
    if filename:
        # Check if file exists to avoid redownload
        if not force and os.path.exists(os.path.join(destination, filename)):
            print(f"✅ Skipping {filename} (Already exists)")
            return
        output_arg = f"-o \"{filename}\""
    else:
        # If no filename given, we can't easily check existence before aria starts,
        # but aria2c usually handles resume/skip if configured right.
        output_arg = ""

    # Handle Civitai Auth
    if "civitai.com" in url and CIVITAI_API_TOKEN:
        if "?" in url:
            url += f"&token={CIVITAI_API_TOKEN}"
        else:
            url += f"?token={CIVITAI_API_TOKEN}"

    print(f"⬇️ Downloading: {filename if filename else url}...")
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M -d "{destination}" {output_arg} "{url}"

print("✅ System Setup Complete.")

Installing System Dependencies...
Selecting previously unselected package libc-ares2:amd64.
(Reading database ... 121689 files and directories currently installed.)
Preparing to unpack .../libc-ares2_1.18.1-1ubuntu0.22.04.3_amd64.deb ...
Unpacking libc-ares2:amd64 (1.18.1-1ubuntu0.22.04.3) ...
Selecting previously unselected package libaria2-0:amd64.
Preparing to unpack .../libaria2-0_1.36.0-1_amd64.deb ...
Unpacking libaria2-0:amd64 (1.36.0-1) ...
Selecting previously unselected package aria2.
Preparing to unpack .../aria2_1.36.0-1_amd64.deb ...
Unpacking aria2 (1.36.0-1) ...
Setting up libc-ares2:amd64 (1.18.1-1ubuntu0.22.04.3) ...
Setting up libaria2-0:amd64 (1.36.0-1) ...
Setting up aria2 (1.36.0-1) ...
Processing triggers for man-db (2.10.2-1) ...
Processing triggers for libc-bin (2.35-0ubuntu3.8) ...
/sbin/ldconfig.real: /usr/local/lib/libtcm.so.1 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind_2_0.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/loca

## Custom Nodes Manager

In [3]:
# --- 2. Install Custom Nodes ---
%cd /content/ComfyUI/custom_nodes

def install_node(url):
    repo_name = url.split('/')[-1].replace('.git', '')
    repo_path = f"/content/ComfyUI/custom_nodes/{repo_name}"

    if os.path.exists(repo_path):
        print(f"✅ {repo_name} already installed. Skipping.")
        return

    print(f"📦 Cloning {repo_name}...")
    !git clone {url}

    # Auto-install requirements
    req_path = f"{repo_path}/requirements.txt"
    if os.path.exists(req_path):
        print(f"   Installing requirements for {repo_name}...")
        !{pip_cmd} install -q -r {req_path}

    # Auto-run install.py
    install_script = f"{repo_path}/install.py"
    if os.path.exists(install_script):
        print(f"   Running install.py for {repo_name}...")
        !{py_cmd} {install_script}

nodes = [
    'https://github.com/ltdrdata/ComfyUI-Manager.git',
    'https://github.com/ltdrdata/ComfyUI-Impact-Pack',
    'https://github.com/ltdrdata/ComfyUI-Impact-Subpack',
    'https://github.com/storyicon/comfyui_segment_anything',
    'https://github.com/Fannovel16/comfyui_controlnet_aux',
    'https://github.com/rgthree/rgthree-comfy',
    'https://github.com/kijai/ComfyUI-KJNodes',
    'https://github.com/1038lab/ComfyUI-RMBG',
    'https://github.com/Kosinkadink/ComfyUI-Advanced-ControlNet',
    'https://github.com/Kosinkadink/ComfyUI-AnimateDiff-Evolved',
    'https://github.com/Kosinkadink/ComfyUI-VideoHelperSuite',
    'https://github.com/JPS-GER/ComfyUI_JPS-Nodes',
    'https://github.com/cubiq/ComfyUI_IPAdapter_plus',
    'https://github.com/ssitu/ComfyUI_UltimateSDUpscale',
    'https://github.com/kijai/ComfyUI-Florence2',
    'https://github.com/sipherxyz/comfyui-art-venture',
    'https://github.com/Kik00/ComfyUI-KikoTools'
]

for node in nodes:
    install_node(node)

print("✅ Custom Nodes Installation Complete.")

/content/ComfyUI/custom_nodes
📦 Cloning ComfyUI-Manager...
Cloning into 'ComfyUI-Manager'...
remote: Enumerating objects: 27813, done.
remote: Counting objects: 100% (517/517), done.
remote: Compressing objects: 100% (246/246), done.
remote: Total 27813 (delta 410), reused 283 (delta 271), pack-reused 27296 (from 4)
Receiving objects: 100% (27813/27813), 124.05 MiB | 24.13 MiB/s, done.
Resolving deltas: 100% (20593/20593), done.
   Installing requirements for ComfyUI-Manager...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 432.7/432.7 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.0/182.0 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.3/22.3 MB 116.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 109.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 88.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 5.7 MB/s eta 0:00:00
📦 Cloning ComfyUI-Impact-Pack...
Cloning i

## Support Models (Ultralytics, SAM, Upscalers)

In [4]:
# --- 3. Download Support Models ---
print("Checking Support Models...")

# 3.1 Ultralytics / Impact Pack Models
bbox_path = '/content/ComfyUI/models/ultralytics/bbox'
segm_path = '/content/ComfyUI/models/ultralytics/segm'
sam_path = '/content/ComfyUI/models/sams'

smart_download('https://huggingface.co/Bingsu/adetailer/resolve/main/face_yolov8m.pt', bbox_path, 'face_yolov8m.pt')
smart_download('https://huggingface.co/Bingsu/adetailer/resolve/main/hand_yolov8s.pt', bbox_path, 'hand_yolov8s.pt')
smart_download('https://huggingface.co/Bingsu/adetailer/resolve/main/person_yolov8m-seg.pt', segm_path, 'person_yolov8m-seg.pt')
smart_download('https://dl.fbaipublicfiles.com/segment_anything/sam_vit_b_01ec64.pth', sam_path, 'sam_vit_b_01ec64.pth')

# 3.2 Upscalers
upscale_path = '/content/ComfyUI/models/upscale_models'
smart_download('https://huggingface.co/shubhdotai/upscaler/resolve/9b9bfba2a119bee0d175427cadb05283a3ed2fb1/4xNMKDSuperscale_4xNMKDSuperscale.pt', upscale_path, '4xNMKDSuperscale_4xNMKDSuperscale.pt')

print("✅ Support Models Ready.")

Checking Support Models...
✅ Skipping face_yolov8m.pt (Already exists)
✅ Skipping hand_yolov8s.pt (Already exists)
✅ Skipping person_yolov8m-seg.pt (Already exists)
✅ Skipping sam_vit_b_01ec64.pth (Already exists)
⬇️ Downloading: 4xNMKDSuperscale_4xNMKDSuperscale.pt...

Download Results:
gid   |stat|avg speed  |path/URI
======+====+===========+=======================================================
9e849b|OK  |    35MiB/s|/content/ComfyUI/models/upscale_models/4xNMKDSuperscale_4xNMKDSuperscale.pt

Status Legend:
(OK):download completed.
✅ Support Models Ready.


## Generation Models (Checkpoints, LoRAs, VAE, ControlNet)

In [5]:
# --- 4. Download Generation Models ---

# Define Paths
ckpt_path = '/content/ComfyUI/models/checkpoints'
lora_path = '/content/ComfyUI/models/loras'
vae_path = '/content/ComfyUI/models/vae'
text_enc_path = '/content/ComfyUI/models/text_encoders'
cnet_path = '/content/ComfyUI/models/controlnet'

# --- 4.1 Checkpoints (Civitai IDs or URLs) ---
civitai_checkpoints = [
    '2071650', # CyberRealistic Pony
    '2442439',
    '2514955',
    '2445746',
    #'2578100', # Dark Beast Z
]
for model_id in civitai_checkpoints:
    # Note: Civitai downloads usually don't need a filename specified, Aria gets it from headers
    smart_download(f'https://civitai.com/api/download/models/{model_id}', ckpt_path)


# --- 4.2 LoRAs ---
smart_download('https://civitai.com/api/download/models/1820705', lora_path)


# --- 4.3 VAEs ---
# Civitai VAE
smart_download('https://civitai.com/api/download/models/2442479', vae_path)
# HuggingFace VAEs (Public)
smart_download("https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.safetensors", vae_path, "vae-ft-mse-840000-ema-pruned.safetensors")
smart_download("https://huggingface.co/Comfy-Org/z_image_turbo/resolve/main/split_files/vae/ae.safetensors", vae_path, "flux-vae.safetensors")


# --- 4.4 Text Encoders ---
smart_download('https://civitai.com/api/download/models/2474529', text_enc_path) # FP8
# smart_download("https://huggingface.co/Comfy-Org/z_image_turbo/resolve/main/split_files/text_encoders/qwen_3_4b.safetensors", text_enc_path, "qwen_3_4b.safetensors")


# --- 4.5 ControlNets ---
# Xinsir ControlNet (Promax) - Note the explicit folder structure for SDXL Union
xinsir_path = f"{cnet_path}/SDXL/controlnet-union-sdxl-1.0"
smart_download("https://huggingface.co/xinsir/controlnet-union-sdxl-1.0/resolve/main/diffusion_pytorch_model_promax.safetensors", xinsir_path, "SDXL_all_in_one_model_promax.safetensors")

print("✅ Generation Models Ready.")

⬇️ Downloading: https://civitai.com/api/download/models/2071650?token=f6309f37a037aa498d45b8e04cfcfbcf...

Download Results:
gid   |stat|avg speed  |path/URI
======+====+===========+=======================================================
704f62|OK  |   190MiB/s|/content/ComfyUI/models/checkpoints/cyberrealisticPony_v127Alt.safetensors

Status Legend:
(OK):download completed.
⬇️ Downloading: https://civitai.com/api/download/models/2442439?token=f6309f37a037aa498d45b8e04cfcfbcf...
 *** Download Progress Summary as of Mon Jan 12 14:17:13 2026 *** 
=
[#ac0b4a 10GiB/11GiB(89%) CN:16 DL:209MiB ETA:5s]
FILE: /content/ComfyUI/models/checkpoints/zImageTurbo_turbo.safetensors
-


Download Results:
gid   |stat|avg speed  |path/URI
======+====+===========+=======================================================
ac0b4a|OK  |   176MiB/s|/content/ComfyUI/models/checkpoints/zImageTurbo_turbo.safetensors

Status Legend:
(OK):download completed.
⬇️ Downloading: https://civitai.com/api/download/models/251

## LIST MODELS

In [6]:
!ls -al ./models/checkpoints/

ls: cannot access './models/checkpoints/': No such file or directory


# `START ComfyUI  & Expose Server (MANUAL)`

## Download Prerequisits

In [ ]:
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb

In [ ]:
!python main.py --dont-print-server

# `START ComfyUI & Expose Server`

## Download Prerequisits

In [9]:
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb

--2026-01-12 14:21:33--  https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
Resolving github.com (github.com)... 140.82.116.4
Connecting to github.com (github.com)|140.82.116.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/cloudflare/cloudflared/releases/download/2025.11.1/cloudflared-linux-amd64.deb [following]
--2026-01-12 14:21:33--  https://github.com/cloudflare/cloudflared/releases/download/2025.11.1/cloudflared-linux-amd64.deb
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/106867604/8a32f7c6-649c-4f0d-806d-e14c19d0786d?sp=r&sv=2018-11-09&sr=b&spr=https&se=2026-01-12T14%3A53%3A28Z&rscd=attachment%3B+filename%3Dcloudflared-linux-amd64.deb&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4d

## CF Tunnel

In [ ]:
import subprocess
import threading
import time
import socket
import urllib.request

def iframe_thread(port):
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()
  print("\nComfyUI finished loading, trying to launch cloudflared (if it gets stuck here cloudflared is having issues)\n")

  p = subprocess.Popen(["cloudflared", "tunnel", "--url", "http://127.0.0.1:{}".format(port)], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
  for line in p.stderr:
    l = line.decode()
    if "trycloudflare.com " in l:
      print("This is the URL to access ComfyUI:", l[l.find("http"):], end='')
    #print(l, end='')


threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

%cd /content/ComfyUI/
!python main.py --dont-print-server --preview-method auto #--lowvram --reserve-vram 0.9 --disable-smart-memory

/content/ComfyUI
[START] Security scan
[DONE] Security scan
## ComfyUI-Manager: installing dependencies done.
** ComfyUI startup time: 2026-01-12 14:23:57.636
** Platform: Linux
** Python version: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
** Python executable: /usr/bin/python3
** ComfyUI Path: /content/ComfyUI
** ComfyUI Base Folder Path: /content/ComfyUI
** User directory: /content/ComfyUI/user
** ComfyUI-Manager config path: /content/ComfyUI/user/__manager/config.ini
** Log path: /content/ComfyUI/user/comfyui.log

Prestartup times for custom nodes:
   0.0 seconds: /content/ComfyUI/custom_nodes/rgthree-comfy
   6.6 seconds: /content/ComfyUI/custom_nodes/ComfyUI-Manager

Checkpoint files will always be loaded safely.
Total VRAM 15095 MB, total RAM 12976 MB
pytorch version: 2.9.1+cu128
xformers version: 0.0.33.post2
Set vram state to: NORMAL_VRAM
Device: cuda:0 Tesla T4 : cudaMallocAsync
Using async weight offloading with 2 streams
Enabled pinned memory 12326.0
working around n

## localtunnel

In [ ]:
# localtunnel
!npm install -g localtunnel

import subprocess
import threading
import time
import socket
import urllib.request

def iframe_thread(port):
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()
  print("\nComfyUI finished loading, trying to launch localtunnel (if it gets stuck here localtunnel is having issues)\n")

  print("The password/enpoint ip for localtunnel is:", urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))
  p = subprocess.Popen(["lt", "--port", "{}".format(port)], stdout=subprocess.PIPE)
  for line in p.stdout:
    print(line.decode(), end='')


threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()
%cd /content/ComfyUI
!python main.py --dont-print-server --lowvram --reserve-vram 0.9